In [ ]:
import os, sys
from pathlib import Path
import numpy as np
import pandas as pd
sys.path.append(str("/home/alessio/dr_sasa_python/build/lib"))
sys.path.append(str("/home/alessio/dr_sasa_python/bindings/python/utils"))
import dr_sasa_py
from structure_parser import *
from utils import convert_to_dataframes

In [ ]:
test_pdb = "/home/alessio/dr_sasa_python/tests/data/pred.pdb"
calc = dr_sasa_py.GenericSASA(probe_radius=1.4)
result = calc.calculate(str(test_pdb), include_matrix=True, print_output=True)
dfs = convert_to_dataframes(result)
overlaps = dfs["overlaps"]
atoms = dfs["atoms"]
residues = dfs["residues"]
overlaps[overlaps["source_residue"].str.startswith("A_")].overlap_area.sum() #A <--- B buried surface (A^2):	791.494
result["intra_bsa_matrix"]["residue_matrix"][:len(residues[residues.chain =="A"].index)].sum() # same as #A <--- B buried surface (A^2):	791.494
result["inter_bsa_matrix"]["A_vs_B"]["matrix"].sum()
##
overlaps[overlaps["source_residue"].str.startswith("B_")].overlap_area.sum() #A ---> B buried surface (A^2):	741.817
result["intra_bsa_matrix"]["residue_matrix"][len(residues[residues.chain =="A"].index):].sum() # same as A ---> B buried surface (A^2):	741.817
result["inter_bsa_matrix"]["B_vs_A"]["matrix"].sum()
##
overlaps.overlap_area.sum()/2 #Interface A/B (A^2):	766.656
result["intra_bsa_matrix"]["atom_matrix"].sum()/2
##
atoms[atoms.chain == "A"].sasa.sum() #Object A complexed surface (A^2):	1198.94 calculation based on atom contact surfaces
residues[residues.chain == "A"].dsasa.sum() # close values (calculated based on standards NACESS sasa values of per residues)
atoms[atoms.chain == "B"].sasa.sum() #Object B complexed surface (A^2):	2165.03
## 
atoms.sasa.sum() #Selected complex surface (A^2):	3363.97
#
overlaps[overlaps["source_residue"].str.startswith("A_")].overlap_area.sum() + atoms[atoms.chain == "A"].sasa.sum() #  Object A uncomplexed surface (A^2):	1990.44
residues[residues.chain == "A"].dsasa.sum()


In [ ]:
test_pdb = "/home/alessio/dr_sasa_python/PRODIGYdataset/1KTZ.pdb"
calc = dr_sasa_py.GenericSASA(probe_radius=1.4)
result = calc.calculate(str(test_pdb), include_matrix=True, print_output=True)

In [ ]:
dfs = convert_to_dataframes(result)
overlaps = dfs["overlaps"]
atoms = dfs["atoms"]
residues = dfs["residues"]

In [ ]:
test_pdb = "/home/alessio/dr_sasa_python/PRODIGYdataset/1FFW.pdb"
calc = dr_sasa_py.SimpleSASA(probe_radius=1.4)
result = calc.calculate(str(test_pdb))
result

In [ ]:
test_pdb = "/home/alessio/dr_sasa_python/PRODIGYdataset/1E6E.pdb"
calc = dr_sasa_py.GenericSASA(probe_radius=1.4)
result = calc.calculate(str(test_pdb), include_matrix=True, print_output=True)
dfs = convert_to_dataframes(result)
overlaps = dfs["overlaps"]
atoms = dfs["atoms"]
residues = dfs["residues"]
overlaps[overlaps["source_residue"].str.startswith("A_")].overlap_area.sum() #A <--- B buried surface (A^2):	791.494
result["intra_bsa_matrix"]["residue_matrix"][:len(residues[residues.chain =="A"].index)].sum() # same as #A <--- B buried surface (A^2):	791.494
result["inter_bsa_matrix"]["A_vs_B"]["matrix"].sum()
##
overlaps[overlaps["source_residue"].str.startswith("B_")].overlap_area.sum() #A ---> B buried surface (A^2):	741.817
result["intra_bsa_matrix"]["residue_matrix"][len(residues[residues.chain =="A"].index):].sum() # same as A ---> B buried surface (A^2):	741.817
result["inter_bsa_matrix"]["B_vs_A"]["matrix"].sum()
##
overlaps.overlap_area.sum()/2 #Interface A/B (A^2):	766.656
result["intra_bsa_matrix"]["atom_matrix"].sum()/2
##
atoms[atoms.chain == "A"].sasa.sum() #Object A complexed surface (A^2):	1198.94 calculation based on atom contact surfaces
residues[residues.chain == "A"].dsasa.sum() # close values (calculated based on standards NACESS sasa values of per residues)
atoms[atoms.chain == "B"].sasa.sum() #Object B complexed surface (A^2):	2165.03
## 
atoms.sasa.sum() #Selected complex surface (A^2):	3363.97
#
overlaps[overlaps["source_residue"].str.startswith("A_")].overlap_area.sum() + atoms[atoms.chain == "A"].sasa.sum() #  Object A uncomplexed surface (A^2):	1990.44
residues[residues.chain == "A"].dsasa.sum()

In [ ]:
residues[residues.chain == "A"].dsasa.sum() - residues[residues.chain == "A"].total_sasa.sum()
residues[residues.chain == "B"].dsasa.sum() - residues[residues.chain == "B"].total_sasa.sum()

In [ ]:
residues.dsasa.sum() - 

In [ ]:
residues[residues.chain == "B"].dsasa.sum()

In [ ]:
residues[residues.chain == "B"].total_sasa.sum()

In [ ]:
result["intra_bsa_matrix"]["residue_matrix"][:21].sum()
result["inter_bsa_matrix"]["B_vs_A"]["matrix"].sum()

In [ ]:
residues[residues.chain == "A"].dsasa.sum()

In [ ]:
residues = dfs["residues"]
residues[residues.chain == "A"].standard_sasa.sum()

Object A complexed surface (A^2):	1201.52
Object B complexed surface (A^2):	2161.36
Object A uncomplexed surface (A^2):	1998.48
Object B uncomplexed surface (A^2):	2905.35
A <--- B buried surface (A^2):	796.957
A ---> B buried surface (A^2):	743.99
Interface A/B (A^2):	770.474

Selected complex surface (A^2):	3363.97
Object A complexed surface (A^2):	1198.94
Object B complexed surface (A^2):	2165.03
Object A uncomplexed surface (A^2):	1990.44
Object B uncomplexed surface (A^2):	2906.85
A <--- B buried surface (A^2):	791.494
A ---> B buried surface (A^2):	741.817
Interface A/B (A^2):	766.656

In [ ]:
test_pdb = "/home/alessio/dr_sasa_python/tests/data/3i40.pdb"
calc = dr_sasa_py.DecoupledSASA(probe_radius=1.4)
result = calc.calculate(str(test_pdb), chains=[['B'], ['C']], print_output=True)

In [ ]:
pd.read_csv("/home/alessio/dr_sasa_python/tests/test_results/3i40.atmasa", sep="\t")
pd.read_csv("/home/alessio/dr_sasa_python/my_analysis", sep="\t")